In [101]:
import pandas as pd
import numpy as np
import math

In [103]:
itr = pd.read_sas("/Users/yeong-injang/Desktop/UofT/BigData/LLCP2023.XPT", chunksize=100000)
df_orig = pd.DataFrame()
for chunk in itr:
    df_orig = pd.concat([chunk[['DIABETE4','_AGE65YR','_RFHYPE6','_RFCHOL3','_RFBMI5','_BMI5','_SMOKER3','STRFREQ_','DRNKANY6','SEXVAR','MENTHLTH','EDUCA','INCOME3','EMPLOY1','_IMPRACE']],df_orig])
df_orig

,DIABETE4,_AGE65YR,_RFHYPE6,_RFCHOL3,_RFBMI5,_BMI5,_SMOKER3,STRFREQ_,DRNKANY6,SEXVAR,MENTHLTH,EDUCA,INCOME3,EMPLOY1,_IMPRACE
400000,3.0,2.0,2.0,1.0,2.0,2650.0,4.0,3.000000e+03,2.0,1.0,1.0,6.0,9.0,7.0,1.0
400001,3.0,1.0,1.0,2.0,2.0,3269.0,4.0,1.000000e+03,2.0,2.0,88.0,6.0,11.0,1.0,1.0
400002,1.0,1.0,1.0,1.0,2.0,2763.0,1.0,1.000000e+03,1.0,2.0,88.0,4.0,6.0,5.0,1.0
400003,1.0,2.0,1.0,1.0,2.0,2509.0,4.0,2.000000e+03,2.0,1.0,88.0,5.0,6.0,7.0,1.0
400004,3.0,1.0,2.0,2.0,2.0,2661.0,4.0,4.667000e+03,2.0,1.0,88.0,5.0,6.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,3.0,1.0,1.0,NaN,2.0,2918.0,1.0,5.397605e-79,2.0,2.0,5.0,4.0,5.0,5.0,1.0
99996,3.0,1.0,2.0,1.0,1.0,2441.0,4.0,5.397605e-79,2.0,1.0,88.0,5.0,77.0,2.0,1.0
99997,3.0,1.0,1.0,1.0,2.0,5721.0,3.0,5.000000e+03,1.0,1.0,88.0,6.0,7.0,1.0,1.0
99998,3.0,2.0,2.0,2.0,1.0,2037.0,4.0,NaN,1.0,2.0,15.0,4.0,6.0,7.0,1.0


In [107]:
df = df_orig.copy()

def chg_elem(x):
    chg_dict = {1:0, 2:1, 9:None}
    return(chg_dict[x])
df = df[df['DIABETE4'].isin([1,3])].reset_index(drop=True) # only yes and no
df = df[df['_AGE65YR'].isin([1,2])] # over 18 only

df['is_diabete'] = df['DIABETE4'].apply(lambda x:1 if x==1 else 0)

df['High_blood_pressure'] = df['_RFHYPE6'].apply(lambda x:chg_elem(x) if math.isnan(x) is False else x) # 1 : yes, 0 : no
df['High_cholesterol'] = df['_RFCHOL3'].apply(lambda x:chg_elem(x) if math.isnan(x) is False else x) # 1 : yes, 0 : no
df['bmi'] = df['_BMI5']

smk = {1:'everyday_smoker',2:'someday_smoker',3:'former_smoker',4:'never_smoked',9:None}

df['smoke'] = df['_SMOKER3'].apply(lambda x:smk[x])
df['exercise'] = df['STRFREQ_'] # strength activity frequency
df['alcohol'] = df['DRNKANY6'].apply(lambda x:{1:1, 2:0, 7:None, 9:None}[x] if math.isnan(x) is False else x) # 1 : yes, 2 : No
df['gender'] = df['SEXVAR'].apply(lambda x:{2:0} if math.isnan(x) is False else x) # 1 : male, 0 : female
df['mental_health'] = df['MENTHLTH'].apply(lambda x: {88:0, 77:None, 99:None}[x]  if math.isnan(x) is False and x>30 else x) # higher - worse

def edu(x):
    if x <= 3:
        result = 'under_highschool'
    elif x <= 5:
        result = 'highschool_grad'
    elif x == 6:
        result = 'college_grad'
    else:
        result = None
    return(result)
    
def employ(x):
    if float(x) < 3:
        result = 1
    elif float(x) < 9:
        result = 0
    else: result = None
    return(result)

rc = {1:'White, Non-Hispanic',
 2:'Black, Non-Hispanic',
 3:'Asian, Non-Hispanic',
 4:'American Indian/Alaskan Native, Non-Hispanic',
 5:'Hispanic',
 6:'Others'}

df['education'] = df['EDUCA'].apply(lambda x:edu(x))
df['income_lv'] = df['INCOME3'].apply(lambda x:x if x<=11 else None)
df['now_working'] = df['EMPLOY1'].apply(lambda x:employ(x)) # 1 : working, 2 : not working
df['race'] = df['_IMPRACE'].apply(lambda x:rc[x])
len(df)

411091

In [108]:
df = df.drop(['DIABETE4','_AGE65YR','_RFHYPE6','_RFCHOL3','_RFBMI5','_BMI5','_SMOKER3','DRNKANY6','SEXVAR','STRFREQ_','MENTHLTH','EDUCA','INCOME3','EMPLOY1','_IMPRACE'], axis=1)
df.isnull().sum()

is_diabete                 0
High_blood_pressure     1444
High_cholesterol       52215
bmi                    35479
smoke                  21054
exercise               13004
alcohol                27449
gender                     0
mental_health           7315
education               1469
income_lv              78739
now_working             6372
race                       0
dtype: int64

In [110]:
df.dropna()['is_diabete'].value_counts()

is_diabete
0    220149
1     39851
Name: count, dtype: int64

In [111]:
df['is_diabete'].value_counts()

is_diabete
0    352226
1     58865
Name: count, dtype: int64